In [2]:
# import keras.callbacks
from music21 import *

import re
import xml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [3]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



FileNotFoundError: [Errno 2] No such file or directory: 'environment.txt'

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

In [4]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [5]:
def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df

In [8]:
# only run if you want to create csv files
import os.path
import os
if not os.path.exists('./data'):
    os.makedirs("./data")
for s in bd:
    fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
    if os.path.isfile(fname):
        continue
    df = convertSongToDataFrame(s.parse().chordify())
    df.to_csv(fname, sep=',')
    
#failed at bwv276.mxl

AttributeError: 'KeySignature' object has no attribute 'tonic'

In [19]:
from functools import reduce
scores = []
import glob, os
os.chdir("C:/Users/errpo/Desktop/projects/project folder/music_prediction/data")
for file in glob.glob("*.csv"):
    scores.append(file)
scoreDf = []

for i in scores:
    score = pd.read_csv(i,usecols=[1,2,3,4,5])
    score.columns = ['keySignature', 'timeSignature', 'chord', 'duration', 'timestamp']
    scoreDf.append(score)

allScores = pd.concat(scoreDf)
keySignatures = pd.DataFrame([allScores['keySignature'].astype('category').cat.codes.unique()], columns=allScores['keySignature'].unique()).to_dict(orient = 'records')
timeSignatures = pd.DataFrame([allScores['timeSignature'].astype('category').cat.codes.unique()], columns=allScores['timeSignature'].unique()).to_dict(orient = 'records')
chords = pd.DataFrame([allScores['chord'].astype('category').cat.codes.unique()], columns=allScores['chord'].unique()).to_dict(orient = 'records')
allScores['keySignature'] = allScores['keySignature'].astype('category').cat.codes
allScores['timeSignature'] = allScores['timeSignature'].astype('category').cat.codes
allScores['chord'] = allScores['chord'].astype('category').cat.codes
allScores.columns = ['keySignature', 'timeSignature', 'chord', 'duration', 'timestamp']

# print(scoreDf[1])
scores = pd.concat(scoreDf[0:10])
# scores = scoreDf[0:10]
testScores = pd.concat(scoreDf[11:12])

avgScoreLength = math.floor(reduce((lambda x,y: x+ len(y)), scoreDf, 0)/len(scoreDf))
#replaceKeySignatures(scores)
keySignatures[0]
def categorize(df):
    df['keySignature']=df['keySignature'].replace(keySignatures[0])
    df['timeSignature']=df['timeSignature'].replace(timeSignatures[0])
    df['chord']=df['chord'].replace(chords[0])

categorize(scores)
categorize(testScores)

scoreDf



[    keySignature timeSignature           chord  duration  timestamp
 0        F major           4/4               I      1.00       0.00
 1        F major           4/4              V6      0.50       1.00
 2        F major           4/4              V6      0.50       1.50
 3        F major           4/4               I      0.50       2.00
 4        F major           4/4               I      0.50       2.50
 5        F major           4/4              vi      0.50       3.00
 6        F major           4/4              vi      0.50       3.50
 7        F major           4/4              I6      0.50       4.00
 8        F major           4/4              I6      0.50       4.50
 9        F major           4/4              IV      0.25       5.00
 10       F major           4/4              IV      0.25       5.25
 11       F major           4/4          IV7642      0.25       5.50
 12       F major           4/4          ii7542      0.25       5.75
 13       F major           4/4   

In [27]:
# copied from http://danielhnyk.cz/predicting-sequences-vectors-keras-using-rnn-lstm/
def _load_data(data, n_prev = 100):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.1):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])
    X_test, y_test = _load_data(df.iloc[ntrn:])

    return (X_train, y_train), (X_test, y_test)

In [28]:
(X_train, y_train), (X_test, y_test) = train_test_split(scores) 

In [29]:
in_out_neurons = 2  
hidden_neurons = 300

model = Sequential()  
model.add(LSTM(in_out_neurons, hidden_neurons, return_sequences=False))  
model.add(Dense(hidden_neurons, in_out_neurons))  
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop")  

TypeError: `LSTM` can accept only 1 positional arguments ('units',), but you passed the following positional arguments: [2, 300]

In [20]:
# values = scores.values
# trainX, trainY = values[:-1, :], values[-1, :]
# testX, testY = values[:-1, :], values[-1, :]
print(scoreDf[13])
trainX = scores.values
trainY = scoreDf[13]
categorize(trainY)
trainY = trainY.values

testX = testScores.values
testY = scoreDf[14]
categorize(testY)
testY = testY.values
values = testScores.values

trainX = trainX.reshape((trainX.shape[0], 1, trainX.shape[1]))
testX = testX.reshape((testX.shape[0], 1, testX.shape[1]))
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

   keySignature timeSignature      chord  duration  timestamp
0       g minor           4/4          i      1.00       0.00
1       g minor           4/4       III6      0.50       1.00
2       g minor           4/4     III763      0.50       1.50
3       g minor           4/4        bVI      0.50       2.00
4       g minor           4/4    bVII743      0.50       2.50
5       g minor           4/4          i      0.50       3.00
6       g minor           4/4     III432      0.50       3.50
7       g minor           4/4      III64      1.00       4.00
8       g minor           4/4        iv6      1.00       5.00
9       g minor           4/4      bVII7      0.25       6.00
10      g minor           4/4        v65      0.25       6.25
11      g minor           4/4      bVII7      0.50       6.50
12      g minor           4/4        III      1.00       7.00
13      g minor           4/4          i      1.00       8.00
14      g minor           4/4          V      0.50       9.00
15      

In [26]:
from matplotlib import pyplot

model = Sequential()
model.add(LSTM(50,input_shape=(None, trainX.shape[2])))
model.add(Dense(5))
model.compile(loss = 'mae', optimizer = 'adam')

history = model.fit(trainX, trainY, epochs=50, batch_size=72, validation_data=(testX, testY), verbose=2, shuffle=False)

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

ValueError: Input arrays should have the same number of samples as target arrays. Found 1013 input samples and 97 target samples.

In [17]:
#LSTM
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [22]:
examples = 20
y_examples = 5
nb_samples = len(data) - examples - y_examples

# np.atleast_2d(data[0:100,:])

In [30]:
input_list = [np.expand_dims(np.atleast_2d(scores[i:examples+i,:]), axis=0) for i in range(nb_samples)]
#input_list = [np.atleast_2d(data[i:examples+i,:]) for i in range(nb_samples)]
input_mat = np.concatenate(input_list, axis=0)

scores

TypeError: unhashable type: 'slice'

In [26]:
target_list = [np.atleast_2d(data[i+examples:examples+i+y_examples,0]) for i in range(nb_samples)]
target_mat = np.concatenate(target_list, axis=0)


NameError: name 'data' is not defined

In [104]:
trials = input_mat.shape[0]
features = input_mat.shape[2]
hidden = 64
model = Sequential()
model.add(LSTM(hidden, input_shape=(examples, features)))
model.add(Dropout(.2))
# model.add(Flatten())
model.add(Dense(y_examples))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='rmsprop')

In [21]:
history = LossHistory()
fittedModel = model.fit(input_mat, target_mat, nb_epoch=100, batch_size=400, callbacks=[history])

NameError: name 'model' is not defined

NameError: name 'fittedModel' is not defined